<a href="https://colab.research.google.com/github/iribeiro191/Ivan_teste_MVP3/blob/main/PUC_RJ_IVAN_RIBEIRO_MVP_Machine_Learning%26Analytics_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="0B3696"> MVP - Machine Learning & Analytics </font>

------
>__Pós-Graduação__: CIÊNCIA DE DADOS E ANALYTICS<br>
>**Sprint**: Machine Learning & Analytics (40530010056_20240_01) <br>
>**Estudante**: IVAN RIBEIRO TEIXEIRA <br>
>**Matrícula**: 4052023001146 <br>
>

----

<font color="0B3696"> **Objetivo** </font>

<span style="font-size:10px;">Este trabalho reflete e consolida os ensinamentos e informações do Sprint: </span>

>
><font color="0B3696">
> Sprint: Machine Learning & Analytics (40530010056_20240_01)
></font>

Curso:
> <font color="0B3696">
> <span style="font-size:10px;"> Pós-Graduação CIÊNCIA DE DADOS E ANALYTICS </span>
> </font>


----
<font color="0B3696" font-face="Ubuntu"> **Análise de Dados** </font> <font color="grey">| SuperLoja </font>


<span style="font-size:10px;"> O dataset escolhido foi do seriado americano **Super Loja**. A escolha de tal dataset deve-se:

*  Por se tratar de informações de vendas, área de negócio que já trabalho e domínio a mais de 20 anos.
*  Por ser uma série que assisti e gostei muito.


Com base no dataset `SuperLoja`, o objetivo é responder algumas perguntas sobre as vendas de produtos e gerar alguns relatórios para tomada de decisão.
<br>

#### **Nota**
>*SuperLoja* lançado em 2015 e transmitida no Brasil por alguns canais fechados de streaming no Amazon Prime Vídeo e Netflix), disponível no portal público do Tableau (https://public.tableau.com/app/learn/sample-data).

</span>


----
<font color="0B3696"> **O Problema** </font> <font color="grey">| Análise de Vendas </font>

A Diretoria da SuperLoja, quer fazer alguns levantamentos para tomada de decisão sobre alguns produtos vendidos e descobrir quais produtos tem poucas vendas e qual tem o menor lucro.

Para resolver esse iremos utilizar machine learning para treinar modelos e iremos realizar um modelo de classificação.



----
<font color="0B3696"> **Parte técnica** </font> <font color="grey">| Configurando o Ambiente </font>


Neste tópico iremos configurar o ambiente instalando e importando as bibliotecas necessárias.

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from tabulate import tabulate
from datetime import datetime
from csv import DictReader

from google.colab import data_table
data_table.enable_dataframe_formatter()

Vamos agora carregar os dados. O dataset utilizado será o SuperLoja, conforme mencionado anteriormente.

In [ ]:
# Carregar os dados
url = "https://raw.githubusercontent.com/iribeiro191/PUC-Rio-machine-learning-analytics-MVP/main/superstore_raw.csv"
df_brasil = pd.read_csv(url, sep=';', on_bad_lines='warn')

Como a Superloja é uma empresa multinacional, vamos filtrar apenas as lojas do Brasil.

In [ ]:
# Filtrando os dados para o Brasil
df = df_brasil[df_brasil['pais'] == 'Brasil'].copy()

Verificar se os dados forma importados com sucesso:

In [ ]:
df

,row_id,pedido,data_pedido,data_envio,tipo_envio,id_cliente,nome_cliente,Segmento,Cidade,Estado,pais,regiao,material,Categoria,sub_categoria,desc_material,Vendas,Quantidade,Desconto,Lucro
6,7,MX-2022-134096,9/28/2022,10/2/2022,Classe padrão,DG-13000,Diogo Goncalves,Varejo,São Paulo,São Paulo,Brasil,Sul,MAT-EN-10000610,Material de escritório,Envelopes,"GlobeWeis Com lacre, Vermelho","561,2",2,0,"213,2"
7,8,MX-2022-134096,9/28/2022,10/2/2022,Classe padrão,DG-13000,Diogo Goncalves,Varejo,São Paulo,São Paulo,Brasil,Sul,TEC-MÁ-10001040,Tecnologia,Máquinas,"Konica Impressora de crachás, Vermelho","3446,4",3,0,"1654,2"
8,9,MX-2022-134096,9/28/2022,10/2/2022,Classe padrão,DG-13000,Diogo Goncalves,Varejo,São Paulo,São Paulo,Brasil,Sul,MAT-MA-10001660,Material de escritório,Materiais,"Elite Estilete, Industrial","973,6",4,0,"194,4"
9,10,MX-2022-134096,9/28/2022,10/2/2022,Classe padrão,DG-13000,Diogo Goncalves,Varejo,São Paulo,São Paulo,Brasil,Sul,TEC-AC-10004490,Tecnologia,Acessórios,"Enermax Roteador, Reciclado","3415,2",2,0,"136,4"
10,11,MX-2022-134096,9/28/2022,10/2/2022,Classe padrão,DG-13000,Diogo Goncalves,Varejo,São Paulo,São Paulo,Brasil,Sul,MAT-FI-10001314,Material de escritório,Fichários,"Wilson Jones Reforço para fichário, Durável","120,6",3,0,"34,8"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,10283,US-2021-145282,4/5/2021,4/8/2021,Primeira classe,LM-17230,Letícia Martins,Varejo,Ponte Nova,Minas Gerais,Brasil,Sul,MAT-ET-10003909,Material de escritório,Etiquetas,"HON Etiquetas de envio, Ajustável","74,16",3,"0,6","-81,84"
10258,10284,US-2021-145282,4/5/2021,4/8/2021,Primeira classe,LM-17230,Letícia Martins,Varejo,Ponte Nova,Minas Gerais,Brasil,Sul,MAT-EN-10001954,Material de escritório,Envelopes,"Jiffy Fita dupla face, Vermelho","62,8",1,"0,6","-64,4"
10259,10285,US-2021-106523,2/11/2021,2/15/2021,Classe padrão,RA-19795,Raissa Azevedo,Pequenas e Medias Empresas,São Miguel dos Campos,Alagoas,Brasil,Sul,MÓV-CA-10004481,Móveis,Cadeiras,"Harbour Creations Cadeira de praia, Vermelho","492,96",3,"0,6","-407,04"
10260,10286,US-2022-168032,11/5/2022,11/9/2022,Classe padrão,BC-11380,Brenda Cardoso,Grande Empresa,Vassouras,Rio de Janeiro,Brasil,Sul,MÓV-ES-10003288,Móveis,Estantes,"Ikea Par, Preto","663,52",2,"0,6","-199,28"


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['row_id'].plot(kind='hist', bins=20, title='row_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Quantidade'].plot(kind='hist', bins=20, title='Quantidade')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('tipo_envio').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['Segmento'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='index', y='Segmento', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['Categoria'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='index', y='Categoria', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['Desconto'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='index', y='Desconto', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

Antes de criar qualquer análise, será necessário realizar alguns tratamentos nos dados.

In [ ]:
# Tratamento de Dados:
cols = ["Lucro","Quantidade"]
df[cols] = df[cols].replace("None", "0")

In [ ]:
# Tratamento de Dados:
cols = ["Lucro","Quantidade"]
df[cols] = df[cols].replace(',', '.')

In [ ]:
# Se a coluna 'Lucro' for do tipo object (string), então substitua ',' por '.' e converta para float
if df['Lucro'].dtype == 'object':
    df['Lucro'] = df['Lucro'].str.replace(',', '.').astype(float)

if df['Quantidade'].dtype == 'object':
    df['Quantidade'] = df['Quantidade'].str.replace(',', '.').astype(int)

**Preparação dos Dados**

Vamos considerar os materiais com vendas no quartil inferior como os "piores" materiais

In [ ]:
df['is_worst'] = df['Quantidade'] <= df['Quantidade'].quantile(0.25)

Agora vamos selecionar as colunas que serão necessárias para descobrir os piores materiais e a pior categoria

In [ ]:
X = df[['Categoria', 'desc_material', 'Quantidade', 'Lucro']]
y = df['is_worst']

Nesta etapa iremos dividir os dados em conjuntos de treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Em seguida, iremos definir o pré-processamento. Foi necessário utilizar a função ``OneHotEncode`` para conventer as colunas categóricas em numéricas. Também foi necessário o uso da função ``StandardScaler`` para normalizar os dados numéricos.

In [ ]:
# Definindo o pré-processamento
# O OneHotEncoder é usado para converter as colunas categóricas em numéricas
# O StandardScaler é usado para normalizar os dados numéricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Quantidade', 'Lucro']),
        ('cat', OneHotEncoder(), ['Categoria'])
    ])

**Criando o pipeline**

*   O ``StandardScaler`` é usado para normalizar os dados numéricos
*   O `RandomForestClassifier` é um modelo de classificação comum


In [ ]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

Nesta etapa iremos definir a grade de hiperparâmetros para a otimização do modelo:

In [ ]:
# Definindo a grade de hiperparâmetros para a otimização
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10]
}

In [ ]:
# Usando o GridSearchCV para otimizar os hiperparâmetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Quantidade',
                                                                          'Lucro']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['Categoria'])])),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [50, 100, 200]},
             scoring='accuracy')

Avaliação do Modelo

In [ ]:
# Avaliando o modelo
y_pred = grid_search.predict(X_test)

Imprimindo o resultado:

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       219
        True       1.00      1.00      1.00       100

    accuracy                           1.00       319
   macro avg       1.00      1.00      1.00       319
weighted avg       1.00      1.00      1.00       319



---
**Pergunta de Negócio**

Agora iremos descobrir qual a categoria menor quantidade vendida e qual o menor lucro:

In [ ]:
# Agrupando os dados por 'Categoria' e calculando a média de 'Quantidade' e 'Lucro'
grouped_df = df.groupby('Categoria')[['Quantidade', 'Lucro']].mean()

# Encontrando a categoria com a menor quantidade média vendida
worst_quantity_category = grouped_df['Quantidade'].idxmin()
worst_quantity_value = grouped_df['Quantidade'].min()

# Encontrando a categoria com o menor lucro médio
worst_profit_category = grouped_df['Lucro'].idxmin()
worst_profit_value = grouped_df['Lucro'].min()

# Calculando os percentuais em relação ao total
total_quantity = df['Quantidade'].sum()
total_profit = df['Lucro'].sum()

worst_quantity_percentage = (worst_quantity_value / total_quantity) * 100
worst_profit_percentage = (worst_profit_value / total_profit) * 100

print(f"A categoria com a menor taxa de quantidade vendida é: {worst_quantity_category} ({worst_quantity_percentage:.2f}%)")
print(f"A categoria com o menor lucro é: {worst_profit_category} ({worst_profit_percentage:.2f}%)")

A categoria com a menor taxa de quantidade vendida é: Material de escritório (0.06%)
A categoria com o menor lucro é: Material de escritório (0.03%)


Agora precisamos descobrir qual o material que é o menos vendido e qual material tem o menor lucro:

In [ ]:
# Agrupando os dados por 'desc_material' e calculando a média de 'Quantidade' e 'Lucro'
grouped_df = df.groupby('desc_material')[['Quantidade', 'Lucro']].mean()

# Encontrando o material com a menor quantidade média vendida
worst_quantity_material = grouped_df['Quantidade'].idxmin()
worst_quantity_value = grouped_df['Quantidade'].min()

# Encontrando o material com o menor lucro médio
worst_profit_material = grouped_df['Lucro'].idxmin()
worst_profit_value = grouped_df['Lucro'].min()

print(f"O material com a menor taxa de quantidade vendida é: {worst_quantity_material}")
print(f"O material com o menor lucro é: {worst_profit_material}")

O material com a menor taxa de quantidade vendida é: Acco Reforço para fichário, Durável
O material com o menor lucro é: Samsung Carregador, Tamanhos variados


-----
<font color="0B3696"> **Conclusão** </font> <font color="grey">

Após realizar todas as análises, podemos responder algumas questões, como por exemplo:
* **Categoria**: A categoria com menor lucro é Material de escritório (0.06%)
* **Materiais**: o Material com menor lucro é Carregador Samsung.

**Desafios**

O maior desafio encontrado, foi o tratamento dos dados. Tive vários problemas durante o treinamento e classificação do modelo devido aos dados.

**Depoimento**

Mesmo eu já estar atuando com soluções de Ciência de Dados, esta SPRINT de “Machine Learning e Analytics” foi desafiadora, pois tive que colocar em prárica todo o ensinamento teórico tanto do curso como na minha vida profissional.

Criar os modelos supervisionados, escolha dos dados de treino e teste além de avaliar os resultados para escolhado dos mehores algorítmos não é tão trivial e simples como eu pensava.

Mais uma vez, consideramo assertiva a escolha do tema do MVP, pois permitiu complementar o aprendizado com os desafios de fazer na prática, colocar a “mão-na-massa”, com muitos, desafios, erros, estudos e acertos.
